In [0]:
## S0 前提事項

- アカウントレベルにて次のグループを作成して、それぞれのグループに所属したユーザーにて Databricks Workspace にログインしてセルを実行する。
    - uc_test__uc_test_01__admin -> アカウント管理者グループ
    -　uc_test_all_read-> 権限付与対象者グループ
- 本ノートブックは、`　uc_test_all_read`の実行を想定している。

In [0]:
## S1 カタログ、スキーマ、テーブルの検証

In [0]:
### S1-1 カタログ、スキーマ、テーブルの作成

In [0]:
%sql
-- 権限不足で確認できない想定
DESCRIBE CATALOG uc_test_01

In [0]:
%sql
-- 権限不足で確認できない想定
DESCRIBE SCHEMA uc_test_01.schema_01

In [0]:
%sql
-- 権限不足で確認できない想定
DESCRIBE TABLE uc_test_01.schema_01.table_01

In [0]:
### S1-2 スキーマのみへの権限付与

In [0]:
%sql
-- `USE CATALOG`権限の不足で確認できない想定
DESCRIBE SCHEMA uc_test_01.schema_01

In [0]:
### S1-3 カタログへの権限付与

In [0]:
%sql
-- 表示される想定
DESCRIBE SCHEMA uc_test_01.schema_01

In [0]:
%sql
-- 表示される想定
DESCRIBE SCHEMA uc_test_01.schema_01

In [0]:
%sql
-- 表示される想定
DESCRIBE TABLE uc_test_01.schema_01.table_01

In [0]:
## S2 関数と Volumes の検証

In [0]:
### S2-1 関数の作成

In [0]:
%sql
-- 権限不足で実行できない想定
SELECT 
  uc_test_01.schema_01.get_rand_num()

In [0]:
%sql
-- 権限不足で表示できない想定
DESCRIBE FUNCTION uc_test_01.schema_01.get_rand_num;

In [0]:
# S2-2 関数への権限付与

In [0]:
%sql
-- 実行できる想定
SELECT 
  uc_test_01.schema_01.get_rand_num()

In [0]:
%sql
-- 表示できる想定
DESCRIBE FUNCTION uc_test_01.schema_01.get_rand_num;

In [0]:
# S2-3 Volumes の作成

In [0]:
%sql
-- 権限不足で表示できない想定
list '/Volumes/uc_test_01/schema_01/volumes_01/delta_01'

In [0]:
# S2-4 Volumes への権限付与

In [0]:
%sql
-- 表示できる想定
list '/Volumes/uc_test_01/schema_01/volumes_01/delta_01'

In [0]:
%sql
-- 表示できる想定
-- ストレージ名が表示されてしまうため、一時的にコメントアウト
-- DESCRIBE VOLUME uc_test_01.schema_01.volumes_01

In [0]:
# S3 動的ビューの検証

In [0]:
# S3-0 事前準備

In [0]:
%sql
-- 権限不足で表示できない想定
DESC SCHEMA uc_test_02.schema_01

In [0]:
%sql
-- 表示できる想定
DESC SCHEMA uc_test_02.schema_dyanamic_view

In [0]:
# S3-1 列レベルのアクセス許可

In [0]:
%sql
SELECT
*
FROM 
    uc_test_02.schema_dyanamic_view.column_level_permissions

In [0]:
### S3-2 行レベルのアクセス許可

In [0]:
%sql
SELECT
*
FROM 
    uc_test_02.schema_dyanamic_view.row_level_permissions

In [0]:
## S4 ユースケース別ワークフローの検証

In [0]:
### S4-1 カタログの払出検証

In [0]:
%sql
DESCRIBE CATALOG uc_test_50

In [0]:
%sql
-- スキーマを作成できることを確認
CREATE SCHEMA IF NOT EXISTS uc_test_50.schema_01;
ALTER SCHEMA uc_test_50.schema_01 OWNER TO uc_test_all_read;

In [0]:
%sql
DESCRIBE SCHEMA uc_test_50.schema_01

In [0]:
### S4-2 スキーマの払出検証

In [0]:
%sql
DESCRIBE SCHEMA uc_test_51.schema_01

In [0]:
%sql
-- テーブルを作成できることを確認
CREATE OR REPLACE TABLE uc_test_51.schema_01.table_01
AS
SELECT
  "efg" AS string_col
  ,3::int AS int_col
  ,CAST('2022-01-01' AS date) AS date_col

In [0]:
%sql
SELECT
*
FROM
  uc_test_51.schema_01.table_01

In [0]:
## S9 リソースの削除